In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
path_output = Path(holo._PATH_OUTPUT).joinpath("share")
paths_gw = [
    "broad-uniform-01-gw_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
    "astro-02-gw_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
    "astro-tight-02-gw_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
]
paths_ph = [
    "broad-uniform-01_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
    "astro-02_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
    # "astro-tight-02-gw_2023-03-03_n10000_s61-81-101_r100_f40_SHARE",
]
fname = "sam_lib.hdf5"
files_gw = []
for path in paths_gw:
    file = path_output.joinpath(path, fname)
    files_gw.append(file)
    print(f"{file.parts[-2]} :: exists = {file.exists()}")
files_ph = []
for path in paths_ph:
    file = path_output.joinpath(path, fname)
    files_ph.append(file)
    print(f"{file.parts[-2]} :: exists = {file.exists()}")    

In [ ]:
fig, axes = plot.figax(ncols=2, scale='lin', sharex=True, sharey=True, xlim=[-20, -12], ylim=[-10, +2])
files_list = [files_gw, files_ph]
titles = ['gw-only', 'gw+phenom']
use_nbins = 5

skip_samps = 3
skip_reals = 3
# skip_samps = 30
# skip_reals = 30

for ii, ax in enumerate(axes):
    ax.set_title(titles[ii], fontsize=10)
    files = files_list[ii]
    handles = []
    names = []
    for fil in files:
        lab = fil.parts[-2].split("_")[0]
        print(lab)
        names.append(lab)
        with h5py.File(fil, 'r') as data:
            lamp = data['fit_lamp']
            plaw = data['fit_plaw']
            nbins = data.attrs['fit_nbins'].tolist()
            lamp = lamp[::skip_samps, ::skip_reals, :].reshape(-1, len(nbins))
            plaw = plaw[::skip_samps, ::skip_reals, :].reshape(-1, len(nbins))

        nb_idx = nbins.index(use_nbins)
        aa = lamp[:, nb_idx]
        bb = plaw[:, nb_idx]

        idx = np.isfinite(aa) & np.isfinite(bb)
        aa = aa[idx]
        bb = bb[idx]
        cc = utils.gamma_strain_to_psd(bb)
        hand = kale.dist2d([aa, cc], ax=ax, scatter=False, hist=False, sigmas=[1, 2, 3], median=False)
        handles.append(hand)
        ax.axvline(-15, color='k', ls='--', alpha=0.5)
        ax.axhline(-13/3, color='k', ls='--', alpha=0.5)

    ax.legend(handles, names, fontsize=8)
    
plt.show()
fname = f"plaw-fits_skip-s{skip_samps}-r{skip_reals}.png"
fname = Path(holo._PATH_OUTPUT).joinpath(fname)
fig.savefig(fname, dpi=400)
print(f"Saved to {fname}, size {utils.get_file_size(fname)}")
        
